### Rastreo de noticias sobre temas de la Ciudad de México que competan o son de interés del Instituto de Planeación de la Ciudad de México

In [83]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import RGBColor
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Pt
import locale
import os
from rapidfuzz import fuzz
import unicodedata

In [84]:
def limpiar_tema(t):
    t = t.replace('%20', ' ')
    palabras = t.split()
    excepciones = ['de', 'la', 'del', 'y', 'en', 'a']  # palabras que no se capitalizan
    palabras_limpias = []

    for i, palabra in enumerate(palabras):
        if i == 0 or palabra.lower() not in excepciones:
            palabras_limpias.append(palabra.capitalize())
        else:
            palabras_limpias.append(palabra.lower())
    
    return ' '.join(palabras_limpias)

In [85]:
#Palabras clave para filtrar solo noticias de la Ciudad de México
palabras_clave_cdmx = ["Foro",
    "Ciudad de México", "CDMX", "Iztapalapa", "Coyoacán", "Cuauhtémoc", "Benito Juárez", 
    "Miguel Hidalgo", "Xochimilco", "Tlalpan", "Gustavo A. Madero", "Venustiano Carranza",
    "Azcapotzalco", "Tláhuac", "Álvaro Obregón", "Milpa Alta", "Magdalena Contreras", "Brugada"
]

In [86]:
# Temas para la búsqueda
temas = ["clara%20brugada",
    "planeación", "infraestructura", "agua", "pgd", "pgot", "metropolitano",
    "zona%20metropolitana",
    "consulta%20pública", "consulta%20indígena", "vivienda",
    "ordenamiento%20territorial", "movilidad", 
    "salud", "gestión%20de%20riesgos", "seguridad%20ciudadana", 
    "participación%20ciudadana", "medio%20ambiente", "cultura", 
    "asentamientos%20irregulares", "ipdp", "congreso%20de%20la%20ciudad%20de%20méxico",
    "transporte%20público", "espacio%20público",
    "patricia%20ramírez%20kuri", "cuidados","función%20pública" 
]

In [87]:
#Palabras clave para filtrar solo noticias de la Ciudad de México
palabras_clave_cdmx = [
    "Ciudad de México", "CDMX", "Iztapalapa", "Coyoacán", "Cuauhtémoc", "Benito Juárez", 
    "Miguel Hidalgo", "Xochimilco", "Tlalpan", "Gustavo A. Madero", "Venustiano Carranza",
    "Azcapotzalco", "Tláhuac", "Álvaro Obregón", "Milpa Alta", "Magdalena Contreras", "Brugada"
]

In [88]:
#Lista de medios relevantes
medios = ["cdhcm",
          "IMER Noticias",
    "CDMX","Imagen Radio 90.5","Aristegui Noticias",
          "88.9 Noticias","W Radio México", "La Octava",
          "Telediario México","capital21.cdmx.gob.mx",
          "CDMX Magacín","Once Noticias",
          "Imagen Televisión",
          "adn40","sdpnoticias",
    'Infobae', 'La Prensa', 'Debate', 'La Crónica de Hoy', 'El Universal',
    'Reforma', 'MVS Noticias', 'SinEmbargo MX', 'Cuarto Poder', 'Reporte Indigo',
    'Eje Central', 'Gobierno de la Ciudad de México', 'La Silla Rota', 'Animal Político', 'La Razón de México',
    'Excélsior', 'Radio Fórmula', 'N+', 'Milenio',
    #'Secretaría de Seguridad Ciudadana de la CDMX',  
    'TV Azteca', 'Quadratín México', 'ContraRéplica',
    'La Jornada', 'Uno TV Noticias', 
    #'Secretaría de Protección Civil CDMX',
    'Ovaciones', 'El Heraldo de México',
    'IECM', 'El Economista', 'RTVE', 'EL PAÍS'
]

#Convertir la lista de medios en una expresión regular
medios_regex = '|'.join(map(re.escape, medios))

In [89]:
def get_notas(tema, periodo):
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/134.0.0.0 Safari/537.36'
        )
    }
    
    url = (
        f'https://news.google.com/search?q={tema}%20ciudad%20de%20mexico'
        f'%20when%3A{periodo}d&hl=es-419&gl=MX&ceid=MX%3Aes-419'
    )
    response = requests.get(url, headers=headers)
    sopa = BeautifulSoup(response.text, 'html.parser')
    
    # AHORA: los bloques de noticias son div.IFHyqb.DeXSAc
    bloques = sopa.select('div.IFHyqb.DeXSAc')

    titulos, urls, fechas, medios_lista = [], [], [], []

    for bloque in bloques:
        # --- TÍTULO ---
        titulo_elem = bloque.find('a', class_='JtKRv')
        titulo = titulo_elem.get_text(strip=True) if titulo_elem else None

        # --- URL ---
        url_nota = None
        if titulo_elem and titulo_elem.get('href'):
            href = titulo_elem.get('href')
            # Suele venir como "./read/..."
            if href.startswith('./'):
                url_nota = 'https://news.google.com' + href[1:]  # quita el punto
            elif href.startswith('http'):
                url_nota = href
            else:
                url_nota = 'https://news.google.com' + href

        # --- MEDIO ---
        medio_elem = bloque.find('div', class_='vr1PYe')
        medio = medio_elem.get_text(strip=True) if medio_elem else None

        # --- FECHA ---
#         #Extraer fecha
        fecha_elem = bloque.find('time')
        fecha = fecha_elem['datetime'] if fecha_elem else None

        titulos.append(titulo)
        urls.append(url_nota)
        fechas.append(fecha)
        medios_lista.append(medio)

    df = pd.DataFrame({
        'titulo': titulos,
        'url': urls,
        'fecha_nota': fechas,
        'medio': medios_lista,
        'fecha_consulta': pd.to_datetime('today').strftime('%Y-%m-%d'),
        'tema': tema
    })

    return df


In [90]:
# #Función para extraer notas
# def get_notas(tema, periodo):
#     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'}
    
#     url = f'https://news.google.com/search?q={tema}%20ciudad%20de%20mexico%20when%3A{periodo}d&hl=es-419&gl=MX&ceid=MX%3Aes-419'
#     response = requests.get(url, headers=headers)
#     sopa = BeautifulSoup(response.text, 'html.parser')
#     resultados = sopa.find_all('article')
    
#     #Listas para almacenar datos
#     titulos, urls, fechas, medios_lista = [], [], [], []

#     for resultado in resultados:
#         #Extraer título
#         titulo_elem = resultado.find('a', class_='JtKRv')
#         titulo = titulo_elem.text.strip() if titulo_elem else None

#         #Extraer URL
#         url_elem = resultado.find('a')
#         url = 'https://news.google.com' + url_elem.get('href')[1:] if url_elem and url_elem.get('href') else None

#         #Extraer fecha
#         fecha_elem = resultado.find('time')
#         fecha = fecha_elem['datetime'] if fecha_elem else None

#         #Extraer medio
#         medio_elem = resultado.find('div', class_='vr1PYe')
#         medio = medio_elem.text.strip() if medio_elem else None

#         #Guardar en listas
#         titulos.append(titulo)
#         urls.append(url)
#         fechas.append(fecha)
#         medios_lista.append(medio)

#     #Crear DataFrame
#     df = pd.DataFrame({
#         'titulo': titulos,
#         'url': urls,
#         'fecha_nota': fechas,
#         'medio': medios_lista,
#         'fecha_consulta': pd.to_datetime('today').strftime('%Y-%m-%d'),
#         'tema': tema
#     })

#     return df

In [91]:
def filtrar_notas(df, umbral=85, medios_preferidos=None):
    if medios_preferidos is None:
        medios_preferidos = []

    df_filtrado = pd.DataFrame(columns=df.columns)
    titulos_grupos = []

    for idx, row in df.iterrows():
        titulo_actual = row['titulo']
        grupo_encontrado = False

        # Buscar si el título se parece a alguno ya registrado
        for grupo in titulos_grupos:
            if any(fuzz.token_set_ratio(titulo_actual, t) >= umbral for t in grupo['titulos']):
                grupo['candidatos'].append(row)
                grupo['titulos'].append(titulo_actual)
                grupo_encontrado = True
                break

        # Si no se parece a ninguno, se crea un nuevo grupo
        if not grupo_encontrado:
            titulos_grupos.append({
                'titulos': [titulo_actual],
                'candidatos': [row]
            })

    # Para cada grupo, elegir el mejor candidato
    for grupo in titulos_grupos:
        candidatos = pd.DataFrame(grupo['candidatos'])
        preferidos = candidatos[candidatos['medio'].isin(medios_preferidos)]

        if not preferidos.empty:
            df_filtrado = pd.concat([df_filtrado, preferidos.iloc[[0]]], ignore_index=True)
        else:
            df_filtrado = pd.concat([df_filtrado, candidatos.iloc[[0]]], ignore_index=True)
#Eliminar notas de Hoy No Circula
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("Hoy No Circula", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("marchas hoy", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("Marchas HOY", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("marchas y bloqueos", case=False, na=False)]


    return df_filtrado.reset_index(drop=True)

In [92]:
periodo =  1 #Definir el periodo de busqueda por días 

#Extraer notas de todos los temas en un solo dataframe
consolidado_notas = pd.concat([get_notas(tema, periodo) for tema in temas], ignore_index=True)

#Filtrar notas de los medios definidos
consolidado_filtrado = consolidado_notas[consolidado_notas['medio'].fillna('').str.contains(medios_regex, regex=True)]

#Filtrar por palabras clave de CDMX
consolidado_final = consolidado_filtrado[consolidado_filtrado['titulo'].fillna('').str.contains('|'.join(palabras_clave_cdmx), regex=True, case=False)]


In [93]:
#Mostrar resultados
print(f'Total de notas encontradas: {len(consolidado_notas)}')
print(f'Notas filtradas por medios: {len(consolidado_filtrado)}')
print(f'Notas finales filtrando por palabras clave: {len(consolidado_final)}')

Total de notas encontradas: 859
Notas filtradas por medios: 271
Notas finales filtrando por palabras clave: 156


In [94]:
# Para hacer el ranking por fecha (más reciente primero)
consolidado_final['fecha_nota'] = pd.to_datetime(consolidado_final['fecha_nota'], errors='coerce')

C:\Users\claud\AppData\Local\Temp\ipykernel_20048\868838740.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consolidado_final['fecha_nota'] = pd.to_datetime(consolidado_final['fecha_nota'], errors='coerce')


In [95]:
# Asegurarnos de que la columna de fecha no tenga zona horaria
consolidado_final['fecha_nota'] = consolidado_final['fecha_nota'].dt.tz_localize(None)

# Ahora podemos guardar en Excel sin problemas
consolidado_final.to_excel("noticias_cdmx.xlsx", index=False)

C:\Users\claud\AppData\Local\Temp\ipykernel_20048\1528448663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consolidado_final['fecha_nota'] = consolidado_final['fecha_nota'].dt.tz_localize(None)


In [96]:
consolidado_final

,titulo,url,fecha_nota,medio,fecha_consulta,tema
1,"Muere Margarita Molina Ríos, madre de Clara Br...",https://news.google.com/read/CBMiswFBVV95cUxQU...,2025-12-01 22:51:07,Infobae,2025-12-02,clara%20brugada
4,"Fallece Margarita Molina Ríos, madre de Clara ...",https://news.google.com/read/CBMizAFBVV95cUxQe...,2025-12-01 22:51:00,El Universal,2025-12-02,clara%20brugada
5,"Fallece Margarita Molina, madre de Clara Brugada",https://news.google.com/read/CBMiiAFBVV95cUxPb...,2025-12-01 23:07:18,Reforma,2025-12-02,clara%20brugada
6,"Falleció Margarita Molina Ríos, madre de la je...",https://news.google.com/read/CBMi0wFBVV95cUxNR...,2025-12-01 22:46:00,La Jornada,2025-12-02,clara%20brugada
7,"Muere Margarita Molina Ríos, Madre de Clara Br...",https://news.google.com/read/CBMivAFBVV95cUxPS...,2025-12-01 22:44:17,N+,2025-12-02,clara%20brugada
...,...,...,...,...,...,...
847,Dua Lipa canta 'Bésame mucho' en su primer con...,https://news.google.com/read/CBMiqwFBVV95cUxOQ...,2025-12-02 04:24:17,Excélsior,2025-12-02,función%20pública
848,"'Un Cuento de Navidad, el musical' en CDMX: Fe...",https://news.google.com/read/CBMiygFBVV95cUxPa...,2025-12-02 00:27:04,Reporte Indigo,2025-12-02,función%20pública
849,¡Dua Lipa y Callum Turner sacaron los prohibid...,https://news.google.com/read/CBMivgFBVV95cUxOb...,2025-12-01 13:59:47,Excélsior,2025-12-02,función%20pública
850,¡Billy Idol recibe sorpresa con mariachi en CD...,https://news.google.com/read/CBMiwAFBVV95cUxOb...,2025-12-01 16:41:21,Excélsior,2025-12-02,función%20pública


Almacenar las notas en el formato de entrega

In [97]:
# Función para agregar hipervínculos con fuente Arial
def add_hyperlink(paragraph, text, url):
    part = paragraph._parent.part
    r_id = part.relate_to(url, "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink", is_external=True)

    hyperlink = OxmlElement("w:hyperlink")
    hyperlink.set(qn("r:id"), r_id)

    run = OxmlElement("w:r")
    rPr = OxmlElement("w:rPr")

    # Fuente Arial
    rFonts = OxmlElement("w:rFonts")
    rFonts.set(qn("w:ascii"), "Arial")
    rFonts.set(qn("w:hAnsi"), "Arial")
    rFonts.set(qn("w:eastAsia"), "Arial")
    rPr.append(rFonts)

    # Color azul
    color = OxmlElement("w:color")
    color.set(qn("w:val"), "0000FF")
    rPr.append(color)

    # Subrayado
    underline = OxmlElement("w:u")
    underline.set(qn("w:val"), "single")
    rPr.append(underline)

    run.append(rPr)

    text_element = OxmlElement("w:t")
    text_element.text = text
    run.append(text_element)

    hyperlink.append(run)
    paragraph._element.append(hyperlink)

In [98]:
# Función de respaldo si locale falla
def fecha_en_espanol(fecha):
    meses = {
        'January': 'enero', 'February': 'febrero', 'March': 'marzo', 'April': 'abril',
        'May': 'mayo', 'June': 'junio', 'July': 'julio', 'August': 'agosto',
        'September': 'septiembre', 'October': 'octubre', 'November': 'noviembre', 'December': 'diciembre'
    }
    fecha_str = fecha.strftime('%d de %B de %Y')
    for en, es in meses.items():
        fecha_str = fecha_str.replace(en, es)
    return fecha_str

In [99]:
def normalizar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')  # quitar acentos
    texto = re.sub(r'\s+', ' ', texto).strip()  # eliminar espacios múltiples
    return texto

def filtrar_notas(df, umbral=85, medios_preferidos=None):
    if medios_preferidos is None:
        medios_preferidos = []

    df = df.copy()
    df['titulo_normalizado'] = df['titulo'].apply(normalizar_texto)

    df_filtrado = pd.DataFrame(columns=df.columns)
    titulos_grupos = []

    for idx, row in df.iterrows():
        titulo_actual = row['titulo_normalizado']
        grupo_encontrado = False

        # Buscar si el título se parece a alguno ya registrado
        for grupo in titulos_grupos:
            if any(
                max(
                    fuzz.token_sort_ratio(titulo_actual, t),
                    fuzz.partial_ratio(titulo_actual, t)
                ) >= umbral
                for t in grupo['titulos']
            ):
                grupo['candidatos'].append(row)
                grupo['titulos'].append(titulo_actual)
                grupo_encontrado = True
                break

        # Si no se parece a ninguno, se crea un nuevo grupo
        if not grupo_encontrado:
            titulos_grupos.append({
                'titulos': [titulo_actual],
                'candidatos': [row]
            })

    # Para cada grupo, elegir el mejor candidato
    for grupo in titulos_grupos:
        candidatos = pd.DataFrame(grupo['candidatos'])
        preferidos = candidatos[candidatos['medio'].isin(medios_preferidos)]

        if not preferidos.empty:
            df_filtrado = pd.concat([df_filtrado, preferidos.iloc[[0]]], ignore_index=True)
        else:
            df_filtrado = pd.concat([df_filtrado, candidatos.iloc[[0]]], ignore_index=True)

    # Eliminar columna auxiliar
    df_filtrado.drop(columns='titulo_normalizado', errors='ignore', inplace=True)
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("Hoy No Circula", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("marchas hoy", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("Marchas HOY", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("Marchas y movilizaciones", case=False, na=False)]

    return df_filtrado.reset_index(drop=True)

In [100]:
#Cargar la tabla de excel
df = pd.read_excel("noticias_cdmx.xlsx")
df['medio'] = df['medio'].fillna('Fuente desconocida')
df['url'] = df['url'].fillna('#')
df['fecha_nota'] = pd.to_datetime(df['fecha_nota'], errors='coerce').dt.strftime('%Y-%m-%d')

def limpiar_tema(t):
    return str(t).strip().capitalize()

df['tema'] = df['tema'].apply(limpiar_tema)
df['tema'] = df['tema'].str.replace('%20', ' ', regex=False)

# Agrupar y deduplicar temas
df['tema_normalizado'] = df['tema'].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_grouped = df.groupby(['fecha_nota', 'medio', 'titulo', 'url']).agg({
    'tema_normalizado': lambda x: ', '.join(sorted(set(t.capitalize() for t in x)))
}).reset_index().rename(columns={'tema_normalizado': 'tema'})

# Filtrar por similitud de títulos
medios_preferidos = ["CDMX","El Universal", "La Jornada", "Milenio", "Reforma", "Excélsior"]
df_grouped = filtrar_notas(df_grouped, umbral=75, medios_preferidos=medios_preferidos)

# Ordenar según prioridad de medios y alfabéticamente
df_grouped['orden_medio'] = df_grouped['medio'].apply(
    lambda x: medios_preferidos.index(x) if x in medios_preferidos else len(medios_preferidos)
)
df_grouped = df_grouped.sort_values(by=['orden_medio', 'medio', 'titulo']).drop(columns='orden_medio')
#Si el medio se llama "CDMX" cambiar por "Gobierno de la Ciudad de México"
df_grouped['medio'] = df_grouped['medio'].replace("CDMX", "Gobierno de la Ciudad de México")
#Cambiar "clara brugada" por "Jefa de Gobierno" y "gestion de riesgos" por "Gestión de Riesgos"
df_grouped['tema'] = df_grouped['tema'].replace({
    "Foro capital y metropolis":"Planeación",
    "Clara brugada": "Jefa de Gobierno",
    "Gestion de riesgos": "Gestión de Riesgos",
    "Espacio publico": "Espacio Público",
    "Participacion ciudadana": "Participación Ciudadana",
    "Planeacion": "Planeación",
    "Congreso de la ciudad de mexico": "Congreso de la Ciudad de México",
    "Transporte publico": "Transporte Público",
}, regex=True)

print(f'Total de notas después de agrupar y filtrar: {len(df_grouped)}')

Total de notas después de agrupar y filtrar: 73


In [101]:
df_grouped

,fecha_nota,medio,titulo,url,tema
11,2025-12-01,El Universal,Clima CDMX: ambiente cálido y cielo medio nubl...,https://news.google.com/read/CBMitgFBVV95cUxNQ...,Gestión de Riesgos
8,2025-12-01,El Universal,"Fallece Margarita Molina Ríos, madre de Clara ...",https://news.google.com/read/CBMizAFBVV95cUxQe...,Jefa de Gobierno
12,2025-12-01,El Universal,Frente Frío 17 en México: ¿qué significan los ...,https://news.google.com/read/CBMi1AFBVV95cUxOU...,Gestión de Riesgos
13,2025-12-01,El Universal,Pide alcaldesa de Xochimilco 15% más de presup...,https://news.google.com/read/CBMihAJBVV95cUxQc...,Asentamientos irregulares
53,2025-12-02,El Universal,"Protestas violentas, tácticas rebasadas: la CD...",https://news.google.com/read/CBMiywFBVV95cUxQe...,Espacio Público
...,...,...,...,...,...
70,2025-12-02,Telediario México,Clima CdMx | Cuál será la temperatura mínima H...,https://news.google.com/read/CBMimAFBVV95cUxPU...,Gestión de Riesgos
71,2025-12-02,Telediario México,"Matan a Javier Alexander, hijo de 'El Barrabás...",https://news.google.com/read/CBMimAFBVV95cUxNN...,Seguridad ciudadana
46,2025-12-01,Telediario México,¡Llegó la navidad! Hasta cuándo estará la verb...,https://news.google.com/read/CBMikwFBVV95cUxNW...,Medio ambiente
47,2025-12-01,W Radio México,Pronóstico clima Ciudad de México: martes 2 de...,https://news.google.com/read/CBMipwFBVV95cUxQS...,"Agua, Gestión de Riesgos"


In [102]:
#Crear un documento de Word
doc = Document()

In [103]:
#Homologar la fuente a Arial
style = doc.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = Pt(10)

style.element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

#Ponerlo en español utf-8
try:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
except:
    try:
        locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')
    except:
        print('No se pudo establecer la localización en español. Usando traducción manual')

fecha_actual = pd.to_datetime('today')
try:
    fecha_formateada = fecha_actual.strftime('%d de %B de %Y')
except:
    fecha_formateada = fecha_en_espanol(fecha_actual)

In [104]:
#Formato de entrega
 
# Título
titulo = doc.add_heading("Monitoreo de Medios y Redes Sociales", level=1)
titulo.alignment = 1  

for run in titulo.runs:
    run.font.name = 'Arial'
    run.font.size = Pt(14)

# Subtítulo
subtitulo1 = doc.add_paragraph("Temas prioritarios de planeación de la Ciudad de México")
subtitulo1.alignment = 1  

for run in subtitulo1.runs:
    run.font.name = 'Arial'
    run.font.size = Pt(12)  

subtitulo2 = doc.add_paragraph(f"Informe con corte al día {fecha_formateada}")
subtitulo2.alignment = 1 

for run in subtitulo2.runs:
    run.font.name = 'Arial'
    run.font.size = Pt(12)  

#Crear tabla
table = doc.add_table(rows=1, cols=4)
table.style = 'Table Grid'

#Variables de la tabla
headers = ["Fecha de la nota", "Fuente", "Idea central", "Temas"]
for i, texto in enumerate(headers):
    p = table.rows[0].cells[i].paragraphs[0]
    run = p.add_run(texto)
    run.font.name = 'Arial'
    run.font.size = Pt(9)
    run._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

for _, row in df_grouped.iterrows():
    row_cells = table.add_row().cells

    #Fecha
    p_fecha = row_cells[0].paragraphs[0]
    run_fecha = p_fecha.add_run(row["fecha_nota"] if pd.notna(row["fecha_nota"]) else "No encontrado")
    run_fecha.font.name = 'Arial'
    run_fecha.font.size = Pt(9)
    run_fecha._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

    #Agregar hipervínculo
    p_medio = row_cells[1].paragraphs[0]
    add_hyperlink(p_medio, row["medio"], row["url"])

    #Título
    p_titulo = row_cells[2].paragraphs[0]
    run_titulo = p_titulo.add_run(row["titulo"])
    run_titulo.font.name = 'Arial'
    run_titulo.font.size = Pt(9)
    run_titulo._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

    #Tema
    p_tema = row_cells[3].paragraphs[0]
    run_tema = p_tema.add_run(row["tema"])
    run_tema.font.name = 'Arial'
    run_tema.font.size = Pt(9)
    run_tema._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

In [105]:
# Guardar
doc.save(f"Monitoreo_Medios_y_Redes_ {fecha_formateada}.docx")